# Implementing TFIDF vectorizer


<font face='georgia'>
    
   <h4><strong>What does tf-idf mean?</strong></h4>

   <p>    
Tf-idf stands for <em>term frequency-inverse document frequency</em>, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.
</p>
    
   <p>
One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.
</p>
    
   <p>
Tf-idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.
</p>
    
</font>

<font face='georgia'>
    <h4><strong>How to Compute:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

<br>
<h4><strong>Example</strong></h4>
<p>

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
</p>
</font>

### Corpus

In [20]:
## SkLearn# Collection of string documents

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

### SkLearn Implementation

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [22]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [23]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [24]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [25]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [26]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.
print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### custom implementation

In [27]:
# Write your code here.
# Make sure its well documented and readble with appropriate comments.
# Compare your results with the above sklearn tfidf vectorizer
# You are not supposed to use any other library apart from the ones given below

from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

In [28]:
def fit(dataset):    
    unique_words = set() # at first we will initialize an empty set
    IDF={} #intitialize an empty dict
    # check if its list type or not
    if isinstance(dataset, (list,)):
        for row in dataset: # for each review in the dataset
            temp=[]  # to count one word only once in a row (if occurs more than once)
            for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                if len(word) < 2:
                    continue
                if word in unique_words and word not in temp: #mark word count for IDF calculatation
                    IDF[word]+=1
                else:
                  if word not in temp:
                      IDF[word]=1
                unique_words.add(word)
                temp.append(word)                           
        IDF_values={i:1+math.log((len(dataset)+1)/(j+1)) for i,j in IDF.items()} #IDF calculation          
        unique_words = sorted(list(unique_words)) 
        vocab = {j:i for i,j in enumerate(unique_words)}
        
        return vocab,IDF_values #returning vocab abd IDF_values
    else:
        print("you need to pass list of sentance")

In [29]:
def transform(dataset,vocab,IDF_values):
    rows = []
    columns = []
    values = []
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(tqdm(dataset)): # for each document in the dataset
            # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
            word_freq = dict(Counter(row.split()))
            # for every unique word in the document
            word_in_row=len(row.split())
            for word, freq in word_freq.items():  # for each unique word in the review.                
                if len(word) < 2:
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                col_index = vocab.get(word, -1) # retreving the dimension number of a word
                # if the word exists
                if col_index !=-1:
                    # we are storing the index of the document
                    rows.append(idx)
                    # we are storing the dimensions of the word
                    columns.append(col_index)
                    # we are storing the frequency of the word
                    tf=freq/word_in_row
                    values.append(tf*IDF_values[word])
        return csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
    else:
        print("you need to pass list of strings")

In [30]:
vocab,IDF_values=fit(corpus) # return vocabs and IDf values
final_ouput=transform(corpus,vocab,IDF_values) # returns final TFIDF  sparse matrix 
final_after_normal=normalize(final_ouput) # normalizing 

100%|██████████| 4/4 [00:00<00:00, 5666.06it/s]


In [31]:
#feature names
print(list(vocab.keys()))
print("\n NO.of features ",len(list(vocab.keys())) )

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

 NO.of features  9


In [32]:
#IDF values
IDF_values

{'and': 1.916290731874155,
 'document': 1.2231435513142097,
 'first': 1.5108256237659907,
 'is': 1.0,
 'one': 1.916290731874155,
 'second': 1.916290731874155,
 'the': 1.0,
 'third': 1.916290731874155,
 'this': 1.0}

In [33]:
#shape of the matrix
final_after_normal.shape

(4, 9)

In [34]:
#ouput of sparse matrix
print(final_after_normal[0])

  (0, 1)	0.4697913855799205
  (0, 2)	0.580285823684436
  (0, 3)	0.3840852409148149
  (0, 6)	0.3840852409148149
  (0, 8)	0.3840852409148149


In [ ]:
#ouput of dense matrix
print(final_after_normal[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


## Implement max features functionality(only top 50):

In [ ]:
# Below is the code to load the cleaned_strings pickle file provided
# Here corpus is of list type

import pickle
with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))

Number of documents in corpus =  746


In [ ]:
# Write your code here.
# Try not to hardcode any values.
# Make sure its well documented and readble with appropriate comments.
def fit(dataset):    
    unique_words = set() # at first we will initialize an empty set
    IDF={} #intitialize an empty dict
    # check if its list type or not
    if isinstance(dataset, (list,)):
        for row in dataset: # for each review in the dataset
            temp=[]
            for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                if len(word) < 2:
                    continue
                if word in unique_words and word not in temp:
                    IDF[word]+=1
                else:
                  if word not in temp:
                      IDF[word]=1
                unique_words.add(word)
                temp.append(word)
        IDF_values={i:1+math.log((len(dataset)+1)/(j+1)) for i,j in IDF.items()}          
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}
        # for TOP N IDF values
        values=list(IDF_values.values())
        values.sort(reverse=True) #sorting the array
        cut=values[49] 
        top_n_IDF={}
        top_n_vocab={}
        k=0  
        for i,j in IDF_values.items(): #storing top 50 , vocab 
          if j>=cut and k<50:
            top_n_IDF[i]=j
            top_n_vocab[i]=k
            k+=1
        return top_n_vocab,top_n_IDF #returning top 50 vocab abd IDF_values
    else:
        print("you need to pass list of sentance")

In [ ]:
def transform(dataset,vocab,IDF_values):
    rows = []
    columns = []
    values = []
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(tqdm(dataset)): # for each document in the dataset
            # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
            word_freq = dict(Counter(row.split()))
            # for every unique word in the document
            word_in_row=len(row.split())
            for word, freq in word_freq.items():  # for each unique word in the review.                
                if len(word) < 2:
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                col_index = vocab.get(word, -1) # retreving the dimension number of a word
                # if the word exists
                if col_index !=-1:
                    # we are storing the index of the document
                    rows.append(idx)
                    # we are storing the dimensions of the word
                    columns.append(col_index)
                    # we are storing the frequency of the word
                    tf=freq/word_in_row
                    values.append(tf*IDF_values[word])
        return csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
    else:
        print("you need to pass list of strings")

In [ ]:
vocab_50,idf_values_50=fit(corpus)   #return top 50 vacabs with idf values
final_output=transform(corpus,vocab_50,idf_values_50) #return final vector
final_after_normal=normalize(final_output, norm='l2', return_norm=False) #after normalize

100%|██████████| 746/746 [00:00<00:00, 96089.14it/s]


In [ ]:
#feature names and IDF values
idf_values_50

{'aimless': 6.922918004572872,
 'artiness': 6.922918004572872,
 'attempting': 6.922918004572872,
 'aye': 6.922918004572872,
 'baby': 6.922918004572872,
 'buffet': 6.922918004572872,
 'conception': 6.922918004572872,
 'constructed': 6.922918004572872,
 'content': 6.922918004572872,
 'distressed': 6.922918004572872,
 'doomed': 6.922918004572872,
 'dozen': 6.922918004572872,
 'drifting': 6.922918004572872,
 'effort': 6.922918004572872,
 'emptiness': 6.922918004572872,
 'existent': 6.922918004572872,
 'fill': 6.922918004572872,
 'fit': 6.922918004572872,
 'florida': 6.922918004572872,
 'gerardo': 6.922918004572872,
 'graphics': 6.922918004572872,
 'highest': 6.922918004572872,
 'insane': 6.922918004572872,
 'massive': 6.922918004572872,
 'messages': 6.922918004572872,
 'muppets': 6.922918004572872,
 'nearly': 6.922918004572872,
 'number': 6.922918004572872,
 'occurs': 6.922918004572872,
 'overdue': 6.922918004572872,
 'owls': 6.922918004572872,
 'person': 6.922918004572872,
 'post': 6.9229

In [ ]:
#shape of the sparse matrix
final_after_normal.shape

(746, 50)

In [ ]:
#output of sparse matrix
print(final_after_normal[0])

  (0, 0)	0.5773502691896257
  (0, 1)	0.5773502691896257
  (0, 2)	0.5773502691896257


In [ ]:
#output of dense matrix
print(final_after_normal[0].toarray())

[[0.57735027 0.57735027 0.57735027 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]
